In [7]:
import pandas as pd
import altair as alt

In [8]:
# large datasets
alt.data_transformers.disable_max_rows()

# load and clean dataa
df = pd.read_csv("combined_nyc_park_crime_stats.csv")
df.columns = df.columns.str.strip().str.upper().str.replace(" ", "_")
df["BOROUGH"] = df["BOROUGH"].astype(str).str.strip().str.title()

# quarter labels
df["QUARTER_STR"] = "Q" + df["QUARTER"].astype(str) + " " + df["YEAR"].astype(str)
df["QUARTER_ORDER"] = df["YEAR"].astype(str) + "Q" + df["QUARTER"].astype(str)

# Define + clean crime cols
crime_columns = [
    "MURDER", "RAPE", "ROBBERY", "FELONY_ASSAULT",
    "BURGLARY", "GRAND_LARCENY", "GRAND_LARCENY_OF_MOTOR_VEHICLE"
]

for col in crime_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# Meltt
df_melted = df.melt(
    id_vars=["BOROUGH", "PARK", "QUARTER", "YEAR", "QUARTER_STR", "QUARTER_ORDER"],
    value_vars=crime_columns,
    var_name="CRIME_TYPE",
    value_name="CRIME_COUNT"
)

# agg
agg = df_melted.groupby(
    ["QUARTER_STR", "QUARTER_ORDER", "CRIME_TYPE", "BOROUGH"]
).agg(CRIME_COUNT=("CRIME_COUNT", "sum")).reset_index().sort_values("QUARTER_ORDER")

# Borough dropdown
borough_dropdown = alt.binding_select(
    options=sorted(agg["BOROUGH"].unique()), name="Select Borough: "
)
borough_select = alt.selection_point(fields=["BOROUGH"], bind=borough_dropdown, value="Bronx")

# Crime type radio
crime_types = sorted(agg["CRIME_TYPE"].unique())
crime_radio = alt.binding_radio(options=crime_types, name="Select Crime Type: ")
crime_select = alt.selection_point(fields=["CRIME_TYPE"], bind=crime_radio, value="ROBBERY")

# filter
filtered = agg

line_chart = alt.Chart(filtered).mark_line(point=True).encode(
    x=alt.X("QUARTER_STR:N", title="Quarter", sort=agg["QUARTER_STR"].unique().tolist()),
    y=alt.Y("CRIME_COUNT:Q", title="Crime Count"),
    color=alt.value('RED'),
    tooltip=["QUARTER_STR", "BOROUGH", "CRIME_TYPE", "CRIME_COUNT"]
).transform_filter(
    borough_select
).transform_filter(
    crime_select
).add_params(
    borough_select,
    crime_select
).properties(
    title="Interactive Dashboard: Crime Trends in NYC Public Parks by Quarter, Borough, and Crime Type (2023–2024)",
    width=1150,
    height=600
)

line_chart.save("interactive_dashboard.html")
line_chart.display()


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.Chart(...)